### Unsupervise data

In [1]:
from tasks.dense_retriever.zero_shot_training.train_data_utils import *

### Supervised Data

In [1]:
def get_args():
    return {'num_layers': 24, 'num_unique_layers': None, 'param_sharing_style': 'grouped', 'hidden_size': 1024, 'ffn_hidden_size': 4096, 'num_attention_heads': 16, 'kv_channels': 64, 'max_position_embeddings': 512, 'make_vocab_size_divisible_by': 128, 'layernorm_epsilon': 1e-05, 'apply_residual_connection_post_layernorm': False, 'openai_gelu': False, 'onnx_safe': None, 'attention_dropout': 0.1, 'hidden_dropout': 0.1, 'weight_decay': 0.1, 'clip_grad': 1.0, 'find_unused_parameters': False, 'batch_size': 2, 'checkpoint_activations': False, 'distribute_checkpointed_activations': False, 'checkpoint_num_layers': 1, 'train_iters': None, 'log_interval': 20, 'exit_interval': None, 'tensorboard_dir': None, 'scaled_upper_triang_masked_softmax_fusion': False, 'scaled_masked_softmax_fusion': False, 'bias_gelu_fusion': False, 'bias_dropout_fusion': False, 'seed': 1234, 'init_method_std': 0.02, 'lr': 2e-05, 'lr_decay_style': 'linear', 'lr_decay_iters': None, 'min_lr': 0.0, 'warmup': 0.01, 'override_lr_scheduler': False, 'use_checkpoint_lr_scheduler': False, 'save': './checkpoints/dualencoder-mss-dpr-large-epochs20-webq', 'save_interval': 5000, 'no_save_optim': False, 'no_save_rng': False, 'load': './checkpoints/dualencoder-mss-dpr-large-epochs20-webq', 'no_load_optim': False, 'no_load_rng': False, 'finetune': False, 'fp16': True, 'apply_query_key_layer_scaling': False, 'attention_softmax_in_fp32': False, 'fp32_allreduce': False, 'hysteresis': 2, 'loss_scale': None, 'loss_scale_window': 1000, 'min_scale': 1, 'fp16_lm_cross_entropy': False, 'model_parallel_size': 1, 'distributed_backend': 'nccl', 'DDP_impl': 'torch', 'local_rank': None, 'lazy_mpu_init': None, 'use_cpu_initialization': False, 'eval_iters': 100, 'eval_interval': 500, 'data_path': None, 'glob': False, 'qa_file_dev': './data/qas/webq-dev.csv', 'qa_file_test': './data/qas/webq-test.csv', 'qa_file_train': None, 'split': '969, 30, 1', 'vocab_file': './bert-vocab/bert-large-uncased-vocab.txt', 'merge_file': None, 'vocab_extra_ids': 0, 'seq_length': 512, 'encoder_seq_length': 512, 'decoder_seq_length': None, 'seq_length_retriever': 256, 'sample_rate': 1.0, 'mask_prob': 0.15, 'short_seq_prob': 0.1, 'mmap_warmup': False, 'num_workers': 2, 'tokenizer_type': 'BertWordPieceLowerCase', 'data_impl': 'infer', 'reset_position_ids': False, 'reset_attention_mask': False, 'eod_mask_loss': False, 'bert_load': './checkpoints/megatron_bert_345m/release/mp_rank_00/model_optim_rng.pt', 'pretrained_dualencoder_load': None, 'evidence_data_path': './data/wikipedia-split/psgs_w100.tsv', 'indexed_evidence_bert_tokenized_data_path': 'data/evidence-wikipedia-indexed-mmap/wikipedia-evidence_text_document', 'indexed_title_bert_tokenized_data_path': 'data/evidence-wikipedia-indexed-mmap/wikipedia-evidence_title_document', 'indexed_evidence_t0_tokenized_data_path': 'data/evidence-wikipedia-indexed-mmap/t0/wikipedia-evidence-t0_text_document', 'indexed_title_t0_tokenized_data_path': 'data/evidence-wikipedia-indexed-mmap/t0/wikipedia-evidence-t0_title_document', 'log_interval_input_data': 100000, 'report_topk_accuracies': [1, 5, 10, 20, 50, 100], 'retriever_score_scaling': True, 'inverse_temperature_multiplier': 1.0, 'art_training': False, 'no_context_embedder_training': False, 'no_query_embedder_training': False, 'disable_retriever_dropout': False, 'index_reload_interval': 500, 'max_training_rank': 1, 'update_retriever': False, 'hf_model_name': 'bigscience/T0_3B', 'verbalizer': ' . Please write a question based on this passage.', 'verbalizer_head': 'Passage: ', 'shard_size': 16, 'initialize_t0_model_tokenizer_evidence': False, 't0_model_in_bf16': False, 'compute_fresh_evidence_embeddings': False, 'faiss_use_gpu': True, 'embedding_path': './embedding-path/psgs_w100-dualencoder-mss-dpr-large-epochs20-webq.pkl', 'match': 'string', 'topk_retrievals': 100, 'save_topk_outputs_path': None, 'indexer_batch_size': 128, 'indexer_log_interval': 1000, 'allow_trivial_doc': False, 'run_indexer': False, 'trec_eval': False, 'task': 'RETRIEVER', 'epochs': 20, 'pretrained_checkpoint': None, 'keep_last': False, 'train_with_neg': True, 'train_data': ['./data/webq/biencoder-webquestions-train.json'], 'valid_data': ['./data/webq/biencoder-webquestions-dev.json'], 'eval_batch_size': 16, 'seq_length_ret': 256, 'train_hard_neg': 7, 'val_av_rank_hard_neg': 5, 'val_av_rank_other_neg': 5, 'rank': 0, 'world_size': 1, 'dynamic_loss_scale': True, 'params_dtype': torch.float16, 'padded_vocab_size': 30592}

### modify the dataset for new data

In [5]:
class OpenRetrievalAbstractDataset(Dataset):
    """Open Retrieval base dataset class."""

    def __init__(self, task_name, dataset_name, datapaths, tokenizer, max_seq_length, evaluate=False,args=None):
        # Store inputs.
        self.args = args
        self.evaluate = evaluate
        self.val_av_rank_hard_neg = args.val_av_rank_hard_neg
        self.val_av_rank_other_neg = args.val_av_rank_other_neg
        self.train_with_neg = args.train_with_neg
        self.train_hard_neg = args.train_hard_neg

        self.task_name = task_name
        self.dataset_name = dataset_name
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        print_rank_0(' > building {} dataset for {}:'.format(self.task_name,
                                                             self.dataset_name))
        # Process the files.
        string = '  > paths:'
        for path in datapaths:
            string += ' ' + path
        print_rank_0(string)
        self.samples = []
        for datapath in datapaths:
            self.samples.extend(self.process_samples_from_single_path(datapath))

        
        if args.sample_rate < 1:  # subsample
            k = int(len(self.samples) * args.sample_rate)
            self.samples = random.sample(self.samples, k)

        print_rank_0('  >> total number of samples: {}'.format(
            len(self.samples)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        raw_sample = self.samples[idx]

        query_ids, query_types, query_pad_mask, ctx_ids, ctx_types, ctx_pad_mask = build_tokens_types_paddings_from_text(
            raw_sample['question'],
            raw_sample['pos_context'],
            self.tokenizer,
            self.max_seq_length)

        if self.evaluate:
            neg_ctx_list = raw_sample['negative_context'][:self.val_av_rank_other_neg] + \
                           raw_sample['hard_negative_context'][:self.val_av_rank_hard_neg]
            neg_ctx_id_list, neg_ctx_types_list = build_token_types_from_context_list(neg_ctx_list,
                                                                                      self.tokenizer,
                                                                                      self.max_seq_length)
        elif self.train_with_neg:
            hard_negative_ctx = raw_sample['hard_negative_context']
            negative_ctx = raw_sample['negative_context']
            if True:  # TODO: fix this or remove this condition
                random.shuffle(hard_negative_ctx)
                random.shuffle(negative_ctx)

            neg_ctx_list = hard_negative_ctx[:self.train_hard_neg]
            # In the Google NQ dataset by DPR paper, there are around more than 50 missing hard negatives in training data.
            # In those cases, substitute hard negatives by simple negatives.
            if len(neg_ctx_list) < self.train_hard_neg:
                neg_ctx_list += negative_ctx[:self.train_hard_neg - len(neg_ctx_list)]

            neg_ctx_id_list, neg_ctx_types_list = build_token_types_from_context_list(neg_ctx_list,
                                                                                      self.tokenizer,
                                                                                      self.max_seq_length)
        else:
            neg_ctx_id_list = None
            neg_ctx_types_list = None

        sample = build_sample(query_ids, query_types, query_pad_mask,
                              ctx_ids, ctx_types, ctx_pad_mask,
                              raw_sample['answers'],
                              neg_ctx_id_list, neg_ctx_types_list,
                              include_neg=self.evaluate or self.train_with_neg)
        return sample

    @staticmethod
    @abstractmethod
    def process_samples_from_single_path(filename):
        """Abstract method that takes a filename and
        returns a list of dataset samples, each sample being a dict of
            {'text': string, 'text': string}
        """
        pass
class Dataset(OpenRetrievalAbstractDataset):

    def __init__(self, name, datapaths, tokenizer, max_seq_length, args=None, evaluate=False):
        super().__init__("open-domain retrieval",
                         name,
                         datapaths,
                         tokenizer,
                         max_seq_length,
                         evaluate=evaluate,args=args)


    def process_samples_from_single_path(self,filename):
        """"Implement abstract method."""
        #args = get_args()
        args = self.args
        print_rank_0(' > Processing {} ...'.format(filename))
        samples = []
        total = 0

        with open(filename, 'r', encoding="utf-8") as f:
            data = json.load(f)
            for row in data:
                question = normalize_question(row['question'])

                # In some datasets, such as Trivia-QA, some positive contexts are also empty
                if len(row['positive_ctxs']) > 0:
                    pos_context = row['positive_ctxs'][0]
                else:
                    continue

                # Hard Negative Contexts
                if len(row['hard_negative_ctxs']) > 0:
                    hard_neg_context = row['hard_negative_ctxs']
                else:
                    hard_neg_context = []

                # Negative Contexts
                if len(row['negative_ctxs']) > 0:
                    neg_context = row['negative_ctxs']
                else:
                    neg_context = []

                if len(hard_neg_context) + len(neg_context) < args.train_hard_neg:
                    continue

                answers = row['answers']
                sample = {'question': question,
                          'pos_context': pos_context,
                          'hard_negative_context': hard_neg_context,
                          'negative_context': neg_context,
                          'answers': answers}
                total += 1
                samples.append(sample)

                if total % 5000 == 0:
                    print_rank_0('  > processed {} so far ...'.format(total))

        print_rank_0(' >> processed {} samples.'.format(len(samples)))
        return samples

In [6]:
class Config:pass
args = Config()
args.val_av_rank_hard_neg = 10
args.val_av_rank_other_neg = 10
args.train_with_neg = 10
args.train_hard_neg = 10
args.sample_rate    =0.4

In [3]:
with open('./data/webq/biencoder-webquestions-train.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

In [27]:
data['Data'][0]['Question']

'Who was the man behind The Chipmunks?'

In [7]:
valid_dataset = Dataset("validation",
['./data/webq/biencoder-webquestions-train.json'],
None,
512,
evaluate=True,args=args)

 > building open-domain retrieval dataset for validation:
  > paths: ./data/webq/biencoder-webquestions-train.json
 > Processing ./data/webq/biencoder-webquestions-train.json ...
 >> processed 2444 samples.
  >> total number of samples: 977


In [10]:
self=valid_dataset
idx = 0

In [12]:
raw_sample

{'question': 'who plays jason gideon',
 'pos_context': {'psg_id': '16520195',
  'text': 'named Donnie Mallick (Arye Gross), which prompts the BAU team to investigate Gideon\'s murder. During the flashbacks focusing on a young version of him for the episode which show him working at the BAU in 1978, he is played by Ben Savage. Jason Gideon Jason Gideon was a fictional character in the CBS crime drama "Criminal Minds", portrayed by Mandy Patinkin. Gideon was a Senior Supervisory Special Agent and the unit chief of the FBI\'s Behavioral Analysis Unit, and has appeared from the series\' pilot episode "Extreme Aggressor," which was originally broadcast on September 22, 2005. He is also portrayed',
  'score': 20.537924,
  'title_score': 0,
  'title': 'Jason Gideon'},
 'hard_negative_context': [{'psg_id': '16520181',
   'text': 'was implied that they have not seen each other very recently. Gideon also has an obsession for birds and bird-watching. The character of Jason Gideon was partially ba

In [11]:
raw_sample = self.samples[idx]

query_ids, query_types, query_pad_mask, ctx_ids, ctx_types, ctx_pad_mask = build_tokens_types_paddings_from_text(
    raw_sample['question'],
    raw_sample['pos_context'],
    self.tokenizer,
    self.max_seq_length)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 raw_sample = self.samples[idx]                                                               │
│   2                                                                                              │
│ ❱ 3 query_ids, query_types, query_pad_mask, ctx_ids, ctx_types, ctx_pad_mask = build_tokens_     │
│   4 │   raw_sample['question'],                                                                  │
│   5 │   raw_sample['pos_context'],                                                               │
│   6 │   self.tokenizer,                                                                          │
│                                                                                                  │
│ /nvme/zhangtianning/projects_local/llm/art/tasks/dense_retriever/supervised_training/train_data_ │
│ utils.py:33 in build_tokens_types_paddings_from_text                                             │
│                                                                                                  │
│    30 │   │   │   │   │   │   │   │   │   │     tokenizer, max_seq_length):                      │
│    31 │   """Build token types and paddings, trim if needed, and pad if needed."""               │
│    32 │                                                                                          │
│ ❱  33 │   query_ids = tokenizer.tokenize(query)                                                  │
│    34 │   query_ids, query_types, query_pad_mask = build_tokens_types_paddings_from_ids(query_   │
│    35 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │     max_se   │
│    36 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │     tokeni   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'tokenize'

In [ ]:
if self.evaluate:
    neg_ctx_list = raw_sample['negative_context'][:self.val_av_rank_other_neg] + \
                   raw_sample['hard_negative_context'][:self.val_av_rank_hard_neg]
    neg_ctx_id_list, neg_ctx_types_list = build_token_types_from_context_list(neg_ctx_list,
                                                                              self.tokenizer,
                                                                              self.max_seq_length)
elif self.train_with_neg:
    hard_negative_ctx = raw_sample['hard_negative_context']
    negative_ctx = raw_sample['negative_context']
    if True:  # TODO: fix this or remove this condition
        random.shuffle(hard_negative_ctx)
        random.shuffle(negative_ctx)

    neg_ctx_list = hard_negative_ctx[:self.train_hard_neg]
    # In the Google NQ dataset by DPR paper, there are around more than 50 missing hard negatives in training data.
    # In those cases, substitute hard negatives by simple negatives.
    if len(neg_ctx_list) < self.train_hard_neg:
        neg_ctx_list += negative_ctx[:self.train_hard_neg - len(neg_ctx_list)]

    neg_ctx_id_list, neg_ctx_types_list = build_token_types_from_context_list(neg_ctx_list,
                                                                              self.tokenizer,
                                                                              self.max_seq_length)
else:
    neg_ctx_id_list = None
    neg_ctx_types_list = None

sample = build_sample(query_ids, query_types, query_pad_mask,
                      ctx_ids, ctx_types, ctx_pad_mask,
                      raw_sample['answers'],
                      neg_ctx_id_list, neg_ctx_types_list,
                      include_neg=self.evaluate or self.train_with_neg)
return sample

In [3]:
print("The dominance ordering {{cite:4f8f559a4b2647def6e5c22a8f895bb9610fd10b}} is defined in the following way.\\nConsider two partitions of the integer {{formula:52949879-7f7d-4f72-8b48-6efdfe8d02c0}} : {{formula:4df4d21f-2211-4ef1-98c2-e78bec14a089}}  and\\n{{formula:988399d8-5bb4-483e-adaa-23ca94affa19}} . Then\\n{{formula:e72e5e80-2d37-4803-8c0f-578b10242eb6}} \\n i.e. the prefix sums of {{formula:26fef996-14ee-469f-abb6-56ff6a03f12b}}  are greater than or equal to\\nthe prefix sums of {{formula:e2d4f490-a378-42ae-9a7e-ae6a48b9e955}} .\\nFrom {{cite:2cedfc1a6c8cb2ab2d588fc26ca023e48c3e2241}}, it is known that the set of partitions of an\\ninteger {{formula:f29d3b3c-86ea-4781-aa9f-c38981972acc}}  equiped with the dominance ordering is a lattice denoted\\nby {{formula:1f7ac1c5-48fa-4a99-9d4c-4c05649b36e3}} . In this paper, Brylawski proposed a dynamical approach to study\\nthis lattice. We will introduce a few notations to explain\\nit intuitively. For more details about integer partitions, we refer\\nto {{cite:37672fc605a1ec41ecd79df884d74fd34ab944ee}}.\\n") 

The dominance ordering {{cite:4f8f559a4b2647def6e5c22a8f895bb9610fd10b}} is defined in the following way.\nConsider two partitions of the integer {{formula:52949879-7f7d-4f72-8b48-6efdfe8d02c0}} : {{formula:4df4d21f-2211-4ef1-98c2-e78bec14a089}}  and\n{{formula:988399d8-5bb4-483e-adaa-23ca94affa19}} . Then\n{{formula:e72e5e80-2d37-4803-8c0f-578b10242eb6}} \n i.e. the prefix sums of {{formula:26fef996-14ee-469f-abb6-56ff6a03f12b}}  are greater than or equal to\nthe prefix sums of {{formula:e2d4f490-a378-42ae-9a7e-ae6a48b9e955}} .\nFrom {{cite:2cedfc1a6c8cb2ab2d588fc26ca023e48c3e2241}}, it is known that the set of partitions of an\ninteger {{formula:f29d3b3c-86ea-4781-aa9f-c38981972acc}}  equiped with the dominance ordering is a lattice denoted\nby {{formula:1f7ac1c5-48fa-4a99-9d4c-4c05649b36e3}} . In this paper, Brylawski proposed a dynamical approach to study\nthis lattice. We will introduce a few notations to explain\nit intuitively. For more details about integer partitions, we refer

In [13]:
print('''
{'model': 'gala120b', 'prompt': "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.\n### Human: What are the key differences between renewable and non-renewable energy sources?\n### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:\n1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.\n2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, and other negative effects.\n3. Cost: Renewable energy sources can be more expensive to initially set up, but they typically have lower operational costs than non-renewable sources.\n4. Reliability: Renewable energy sources are often more reliable and can be used in more remote locations than non-renewable sources.\n5. Flexibility: Renewable energy sources are often more flexible and can be adapted to different situations and needs, while non-renewable sources are more rigid and inflexible.\n6. Sustainability: Renewable energy sources are more sustainable over the long term, while non-renewable sources are not, and their depletion can lead to economic and social instability.\n### Human: Give me the derivation of Evidence Lower Bound (ELBO) algorithm step by step\n### Assistant:", 'temperature': 0.7, 'max_new_tokens': 512, 'stop': '###', 'stop_token_ids': None, 'echo': False}

''')


{'model': 'gala120b', 'prompt': "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
### Human: What are the key differences between renewable and non-renewable energy sources?
### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:
1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.
2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions,

In [7]:
print("A partition of the integer {{formula:2565c197-8a60-4503-af8b-4efa1ca8d8ba}}  is a {{formula:c439eeb8-812b-497a-9a72-8407d90cfbfb}} -tuple {{formula:c49dea42-c345-470a-b733-41e10b7b3037}} \nof positive integers such that {{formula:fc81d36e-9235-4cb3-b00e-625611d221f7}}  and\n{{formula:9fa6291b-d38e-4c86-b986-61fd4ca91dd7}}  for all {{formula:4017aef1-e3ff-4e75-b078-e14c599b7c24}}  between 1 and {{formula:27266042-85d6-4805-99f2-fe0ac94aa4a2}}  (with the\nassumption that {{formula:7a491f87-bc22-4bae-856c-0d0d715cf4c0}} ).\nThe Ferrers diagram of a partition {{formula:c4cac4f6-8a2a-4edc-abcc-3d8a68bd7fc4}}  is a\ndrawing of {{formula:0d55a55c-ad9f-4e0f-b32a-2f9d2a42ac81}}  on {{formula:93c7a1d6-30f3-4cb1-bab3-3033de5484fe}}  adjacent columns such that the {{formula:e60ce57e-f521-4749-9fd3-cab0e609b26c}} -th column\nis a pile of {{formula:8c4b07e6-059e-4d01-81e2-26fb6a64b0cf}}  packed squares (called grains). See\nFigure\u00a0REF  for examples.\n")

A partition of the integer {{formula:2565c197-8a60-4503-af8b-4efa1ca8d8ba}}  is a {{formula:c439eeb8-812b-497a-9a72-8407d90cfbfb}} -tuple {{formula:c49dea42-c345-470a-b733-41e10b7b3037}} 
of positive integers such that {{formula:fc81d36e-9235-4cb3-b00e-625611d221f7}}  and
{{formula:9fa6291b-d38e-4c86-b986-61fd4ca91dd7}}  for all {{formula:4017aef1-e3ff-4e75-b078-e14c599b7c24}}  between 1 and {{formula:27266042-85d6-4805-99f2-fe0ac94aa4a2}}  (with the
assumption that {{formula:7a491f87-bc22-4bae-856c-0d0d715cf4c0}} ).
The Ferrers diagram of a partition {{formula:c4cac4f6-8a2a-4edc-abcc-3d8a68bd7fc4}}  is a
drawing of {{formula:0d55a55c-ad9f-4e0f-b32a-2f9d2a42ac81}}  on {{formula:93c7a1d6-30f3-4cb1-bab3-3033de5484fe}}  adjacent columns such that the {{formula:e60ce57e-f521-4749-9fd3-cab0e609b26c}} -th column
is a pile of {{formula:8c4b07e6-059e-4d01-81e2-26fb6a64b0cf}}  packed squares (called grains). See
Figure REF  for examples.



In [21]:
import json

In [23]:
json.dumps({'question': '1-dim representations of the affine Hecke algebra for $G = \text{SL}_2$', 'answer': 'geometric methods in representation theory of hecke algebras and quantum   groups', 'positive_ctxs': [{'title': 'geometric methods in representation theory of hecke algebras and quantum   groups', 'abstract': 'these lectures given in montreal in summer 1997 are mainly based on, and form a condensed survey of, the book by n. chriss and v. ginzburg: `representation theory and complex geometry', birkhauser 1997.   various algebras arising naturally in representation theory such as the group algebra of a weyl group, the universal enveloping algebra of a complex semisimple lie algebra, a quantum group or the iwahori-hecke algebra of bi-invariant functions (under convolution) on a p-adic group, are considered.   we give a uniform geometric construction of these algebras in terms of homology of an appropriate "steinberg-type" variety z (or its modification, such as k-theory or elliptic cohomology of z, or an equivariant version thereof). we then explain how to obtain a complete classification of finite dimensional irreducible representations of the algebras in question, using our geometric construction and perverse sheaves methods.   similar techniques can be applied to other algebras, e.g. the double-affine hecke algebras, elliptic algebras, quantum toroidal algebras.'}], 'negative_ctxs': [], 'hard_negative_ctxs': []})

SyntaxError: invalid syntax (2528223171.py, line 1)

### Lets check the Tokenized_Knowledge_Dataset

In [11]:
args.indexed_evidence_bert_tokenized_data_path = "./data/unarXive/evidence-unarxiv-indexed-mmap/bert/unarxiv-keyword-evidence-bert_text_document"
args.indexed_title_bert_tokenized_data_path    = "./data/unarXive/evidence-unarxiv-indexed-mmap/bert/unarxiv-keyword-evidence-bert_title_document"

In [9]:
_ = _build_tokenizer(args)

> building BertWordPieceLowerCase tokenizer with 0 extra vocab ids...
 > padded vocab (size: 30524) with 68 dummy tokens (new size: 30592)


In [18]:
from megatron.data.pretokenized_evidence import Tokenized_Knowledge_Dataset
bert_knowledge = Tokenized_Knowledge_Dataset((args.indexed_evidence_bert_tokenized_data_path, 
                                             args.indexed_title_bert_tokenized_data_path),
        args.data_impl,
        get_tokenizer(),args.seq_length_retriever, mode='bert.both_query_and_passage')

 >>>>> will collect knowledge via [bert.both_query_and_passage] way
 >>>>> will collect knowledge information from  [('./data/unarXive/evidence-unarxiv-indexed-mmap/bert/unarxiv-keyword-evidence-bert_text_document', './data/unarXive/evidence-unarxiv-indexed-mmap/bert/unarxiv-keyword-evidence-bert_title_document')]
  >> total collect 154646 passages


In [19]:
tokenizer = get_tokenizer()

In [22]:
bert_knowledge[0].keys()

dict_keys(['row_id', 'title_text_ids'])

In [20]:
tokenizer.decode(bert_knowledge[0]['title_text_ids'])

'[CLS] what is minkowskian [SEP] we demonstrate that assuming the " discrete " vacuum geometry in the minkowskian higgs model with vacuum bps monopole solutions can justify the dirac fundamental quantization of that model. the important constituent of this quantization is getting various rotary effects, including collective solid rotations inside the physical bps monopole vacuum, and just assuming the " discrete " vacuum geometry seems to be that thing able to justify these rotary effects. more precisely, assuming the " discrete " geometry for the appropriate vacuum manifold implies the presence of thread topological defects ( side by side with point hedgehog topological defects and walls between different topological domains ) inside this manifold in the shape of specific ( rectilinear ) threads : gauge and higgs fields located in the spatial region intimately near the axis $ z $ of the chosen ( rest ) reference frame. this serves as the source of collective solid rotations proceeding

# lets start tracing - Supervised

### load config, data and model

In [1]:
from tasks.dense_retriever.supervised_training.run import *

from tasks.dense_retriever.supervised_training.train_data_utils import Dataset as dataset_cls

import torch

from megatron.global_vars import _build_tokenizer,_set_tensorboard_writer,_set_timers
from megatron.initialize import initialize_megatron

args = get_args()
_ = _build_tokenizer(args)
_set_tensorboard_writer(args)
_set_timers()


initialize_megatron()

In [8]:
def train_valid_datasets_provider():
        args = get_args()
        tokenizer = get_tokenizer()

        train_dataset = dataset_cls("training",
                                    args.train_data,
                                    tokenizer,
                                    args.seq_length_retriever,
                                    evaluate=False)
        valid_dataset = dataset_cls("validation",
                                    args.valid_data,
                                    tokenizer,
                                    args.seq_length_retriever,
                                    evaluate=True)
        return train_dataset, valid_dataset

def model_provider():
    args = get_args()
    print_rank_0('building retriever model for {} ...'.format(args.task))
    model = dualencoder_model_provider(only_context_model=False,
                                       only_query_model=False)
    return model

def single_dataset_provider(datapath):
    args = get_args()
    tokenizer = get_tokenizer()
    name_from_datapath = datapath[0].split('/')[-1].split('.')[0]

    return dataset_cls(name_from_datapath,
                       datapath,
                       tokenizer,
                       args.seq_length_retriever,
                       evaluate=True)

def distributed_metrics_func_provider(datapath):
    return accuracy_func_provider(single_dataset_provider, datapath)

In [ ]:
# train(train_valid_datasets_provider,
#       model_provider,
#       end_of_epoch_callback_provider=distributed_metrics_func_provider)

In [9]:
import torch

In [10]:
# train_dataset, valid_dataset = train_valid_datasets_provider()
args=args = get_args()
tokenizer = get_tokenizer()
train_dataset = dataset_cls("validation",
                            args.train_data,
                            tokenizer,
                            args.seq_length_retriever,
                            evaluate=False)
valid_dataset = dataset_cls("validation",
                            args.valid_data,
                            tokenizer,
                            args.seq_length_retriever,
                            evaluate=True)

---------> use fixed config <----------
 > building open-domain retrieval dataset for validation:
  > paths: ./data/webq/biencoder-webquestions-train.json
---------> use fixed config <----------
 > Processing ./data/webq/biencoder-webquestions-train.json ...
 >> processed 2447 samples.
---------> use fixed config <----------
  >> total number of samples: 2447
---------> use fixed config <----------
 > building open-domain retrieval dataset for validation:
  > paths: ./data/webq/biencoder-webquestions-dev.json
---------> use fixed config <----------
 > Processing ./data/webq/biencoder-webquestions-dev.json ...
 >> processed 276 samples.
---------> use fixed config <----------
  >> total number of samples: 276


In [11]:
train_dataset[0].keys()

dict_keys(['query', 'query_mask', 'query_types', 'query_pad_mask', 'context', 'context_mask', 'context_types', 'context_pad_mask', 'reference', 'neg_context', 'neg_context_types', 'neg_context_mask'])

In [12]:
from tasks.dense_retriever.supervised_training.train_dense_retriever import _build_train_valid_dataloaders,setup_model_and_optimizer

In [13]:
from megatron import mpu

In [14]:
train_dataloader, valid_dataloader = _build_train_valid_dataloaders(train_dataset, valid_dataset)

---------> use fixed config <----------
building train and validation dataloaders ...


In [15]:
model, optimizer, lr_scheduler = setup_model_and_optimizer(model_provider)

---------> use fixed config <----------
---------> use fixed config <----------
building retriever model for RETRIEVER ...
---------> use fixed config <----------
building DualEncoderModel...
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
--------

dsw-23976-7d79d88dc8-j998v:250853:250853 [0] NCCL INFO Bootstrap : Using eth0:10.246.119.4<0>
dsw-23976-7d79d88dc8-j998v:250853:250853 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
dsw-23976-7d79d88dc8-j998v:250853:250853 [0] NCCL INFO cudaDriverVersion 11040
NCCL version 2.14.3+cuda11.7
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO Failed to open libibverbs.so[.1]
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO NET/Socket : Using [0]eth0:10.246.119.4<0>
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO Using network Socket
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO NCCL_MAX_NCHANNELS set by environment to 2.
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO NCCL_MIN_NCHANNELS set by environment to 2.
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO Channel 00/02 :    0
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO Channel 01/02 :    0
dsw-23976-7d79d88dc8-j998v:250853:250954 [0] NCCL INFO

In [122]:
from megatron.model import get_params_for_weight_decay_optimization

In [123]:
param_groups = get_params_for_weight_decay_optimization(model)

In [128]:
param_groups[0]['params'][0].name

## go into train

In [17]:
fetcher= iter(train_dataloader)
batch = next(fetcher)

In [20]:
batch['query'].shape

torch.Size([2, 256])

### go into `_cross_entropy_forward_step`

```
losses_dict, skipped_iter = train_step(forward_step, batch, model,optimizer, lr_scheduler)
--------------
loss, loss_reduced = forward_step_func(data_iterator, model)
--------------
_cross_entropy_forward_step(batch, model)
```

In [22]:
from tasks.dense_retriever.supervised_training.train_dense_retriever import process_batch

In [23]:
query_tokens, query_mask, query_types, query_pad_mask, \
    context_tokens, context_mask, context_types, context_pad_mask, \
    neg_context_tokens, neg_context_mask, neg_context_types, \
    reference = process_batch(batch)

In [26]:
local_batch_size = query_tokens.shape[0]

# Text representation of query and context
query_list, context_list = [], []
for i in range(local_batch_size):
    query_list.append(tokenizer.decode(query_tokens[i].tolist()))
    context_list.append(tokenizer.decode(context_tokens[i].tolist()))

if neg_context_tokens is not None:
    context_tokens = torch.cat([context_tokens, neg_context_tokens])
    context_mask = torch.cat([context_mask, neg_context_mask])
    context_types = torch.cat([context_types, neg_context_types])

In [34]:
query_list

['[CLS] what inventions did leonardo da vinci made [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [54]:
context_list

["[CLS] science and inventions of leonardo da vinci [SEP] itself would have rotated in the opposite direction to the rotor. while he designed a number of man powered flying machines with mechanical wings that flapped, he also designed a parachute and a light hang glider which could have flown. the viola organista was an experimental musical instrument invented by leonardo da vinci. it was the first bowed keyboard instrument ( of which any record has survived ) ever to be devised. leonardo's original idea, as preserved in his notebooks of 1488 – 1489 and in the drawings in the codex atlanticus, was to use one or more wheels, continuously rotating, each of which [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [30]:
query_logits, context_logits = model(query_tokens,query_mask,query_types,
                                   context_tokens,context_mask,context_types)

In [97]:
from megatron.model.bert_model import bert_attention_mask_func, bert_position_ids


False

In [112]:
llm.add_pooler

False

In [115]:
llm.add_decoder

False

In [116]:
context_logits.shape

torch.Size([16, 1024])

In [111]:
with torch.no_grad():
    themodel = model.module.module.query_model
    input_ids, attention_mask, tokentype_ids = query_tokens,query_mask,query_types
    extended_attention_mask = attention_mask.unsqueeze(1)
    position_ids = bert_position_ids(input_ids)
    llm = themodel.language_model
    enc_input_ids = input_ids;
    enc_position_ids = position_ids;
    enc_attention_mask = extended_attention_mask;
    tokentype_ids=tokentype_ids;
    dec_input_ids=None;dec_position_ids=None;dec_attn_mask=None;enc_dec_attn_mask=None;
    layer_past=None;get_key_value=False;pooling_sequence_index=0;
    enc_hidden_states=None;output_enc_hidden=False;sim_scores=None;
    print(enc_input_ids.shape)
    print(enc_position_ids.shape)
    print(tokentype_ids.shape)
    enc_embedding_output = llm.embedding(enc_input_ids,
                                      enc_position_ids,
                                      tokentype_ids=tokentype_ids)
    print(enc_embedding_output.shape)
    if enc_hidden_states is None:
        encoder_output = llm.encoder(enc_embedding_output,enc_attention_mask,layer_past=layer_past,get_key_value=get_key_value)
    else:
        encoder_output = enc_hidden_states.to(enc_embedding_output.dtype)
    print(encoder_output.shape)
    if llm.add_pooler:
        pooled_output = llm.pooler(encoder_output,pooling_sequence_index)
        print(pooled_output.shape)
    #lm_output = llm(input_ids,position_ids,extended_attention_mask,tokentype_ids=tokentype_ids)
    #query_logits = themodel(query_tokens,query_mask,query_types)

torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256, 1024])
torch.Size([2, 256, 1024])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:28                                                                                   │
│                                                                                                  │
│   25 │   print(encoder_output.shape)                                                             │
│   26 │   if llm.add_pooler:                                                                      │
│   27 │   │   pooled_output = llm.pooler(encoder_output,pooling_sequence_index)                   │
│ ❱ 28 │   print(pooled_output.shape)                                                              │
│   29 │   #lm_output = llm(input_ids,position_ids,extended_attention_mask,tokentype_ids=tokent    │
│   30 │   #query_logits = themodel(query_tokens,query_mask,query_types)                           │
│   31                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pooled_output' is not defined

In [38]:
from tasks.dense_retriever.supervised_training.train_dense_retriever import get_group_world_size_rank

In [39]:
group, rank, world_size = get_group_world_size_rank()
global_batch_size = world_size * local_batch_size  # recall we assert that model_parallel_size == 1

In [43]:
all_query_logits.shape

torch.Size([2, 1024])

In [ ]:
opt

In [42]:
all_query_logits = query_logits
all_context_logits = context_logits

retrieval_scores = torch.matmul(all_query_logits,torch.transpose(all_context_logits, 0, 1))

In [46]:
from tasks.dense_retriever.supervised_training.train_dense_retriever import F,reduce_losses

In [60]:
query_logits.shape

torch.Size([2, 1024])

In [61]:
context_logits.shape

torch.Size([16, 1024])

In [53]:
max_idxs

tensor([12,  5], device='cuda:0')

In [58]:
all_query_logits.shape

torch.Size([2, 1024])

In [85]:
for key, val in batch.items():
    try:
        print(f"{key:30s} --> {val.shape}")
    except:
        print(f"{key:30s}")

query                          --> torch.Size([2, 256])
query_mask                     --> torch.Size([2, 256, 256])
query_types                    --> torch.Size([2, 256])
query_pad_mask                 --> torch.Size([2, 256])
context                        --> torch.Size([2, 256])
context_mask                   --> torch.Size([2, 256, 256])
context_types                  --> torch.Size([2, 256])
context_pad_mask               --> torch.Size([2, 256])
reference                     
neg_context                    --> torch.Size([14, 256])
neg_context_types              --> torch.Size([14, 256])
neg_context_mask               --> torch.Size([14, 256, 256])


In [47]:
labels = torch.arange(global_batch_size).long().cuda()

# Cross-entropy loss.
softmax_scores = F.log_softmax(retrieval_scores, dim=1)

loss = F.nll_loss(softmax_scores, labels, reduction='mean')

max_score, max_idxs = torch.max(softmax_scores, 1)
correct_predictions_count = (max_idxs == labels).sum().float()

# Reduce loss for logging.
reduced_loss = reduce_losses([loss, correct_predictions_count])


In [49]:
correct_predictions_count

tensor(0., device='cuda:0')

In [48]:
reduced_loss

tensor([26.7394,  0.0000], device='cuda:0')

# lets start tracing - Unsupervised

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["NCCL_DEBUG"] = "WARN"
os.environ["LOCAL_RANK"] = "0"
from tasks.dense_retriever.supervised_training.run import *
from tasks.dense_retriever.zero_shot_training.train_data_utils import OpenQADataset as dataset_cls

import torch
from megatron.global_vars import _build_t0_tokenizer,_build_t0_model,_load_wikipedia_evidence,_build_tokenizer,_set_tensorboard_writer,_set_timers
from megatron.initialize import initialize_megatron

import torch
from megatron.global_vars import _build_t0_tokenizer,_build_t0_model,_load_wikipedia_evidence,_build_tokenizer,_set_tensorboard_writer,_set_timers
from megatron.initialize import initialize_megatron
from megatron.global_vars import _build_knowledge_pool_in_bert,_build_knowledge_pool_in_t0

args = get_args()

---------> use fixed config <----------


In [2]:
_ = _build_tokenizer(args)
_ = _build_t0_tokenizer(args)

> building BertWordPieceLowerCase tokenizer with 0 extra vocab ids...
 > padded vocab (size: 30524) with 68 dummy tokens (new size: 30592)


In [5]:
aaa = get_bert_tokenizer()

In [6]:
tensor = torch.load("debug.pt")

In [7]:
tensor.shape

torch.Size([128, 256])

In [18]:
from transformers import BertTokenizer as HFBertTokenizer


hf_bert_tokenizer = HFBertTokenizer.from_pretrained("bert-large-uncased")
hf_bert_tokenizer.decode(tensor[2], skip_special_tokens=True)

"id elements are short interspersed elements ( sines ) found in high copy number in many rodent genomes. bc1 rna, an id - related transcript, is derived from the single copy bc1 rna gene. the bc1 rna gene has been shown to be a master gene for id element amplification in rodent genomes. id elements are dispersed through a process termed retroposition. the retroposition process involves a number of potential regulatory steps. these regulatory steps may include transcription in the appropriate tissue, transcript stability, priming of the rna transcript for reverse transcription and integration. this study focuses on priming of the rna transcript for reverse transcription. bc1 rna gene transcripts are shown to be able to prime their own reverse transcription in an efficient intramolecular and site - specific fashion. this self - priming ability is a consequence of the secondary structure of the 3'- unique region. the observation that a gene actively amplified throughout rodent evolution m

In [2]:
_ = _build_knowledge_pool_in_bert(args)
_ = _build_knowledge_pool_in_t0(args)

> building BertWordPieceLowerCase tokenizer with 0 extra vocab ids...
 > padded vocab (size: 30524) with 68 dummy tokens (new size: 30592)
 >>>>> will collect knowledge via [bert.only_passage] way
 >>>>> will collect knowledge information from  [('./data/evidence-wikipedia-indexed-mmap/bert/wikipedia-evidence-bert_text_document', './data/evidence-wikipedia-indexed-mmap/bert/wikipedia-evidence-bert_title_document')]
Dataset does not exist: ./data/evidence-wikipedia-indexed-mmap/bert/wikipedia-evidence-bert_text_document
Path should be a basename that both .idx and .bin can be appended to get full filenames.
Dataset does not exist: ./data/evidence-wikipedia-indexed-mmap/bert/wikipedia-evidence-bert_title_document
Path should be a basename that both .idx and .bin can be appended to get full filenames.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 _ = _build_tokenizer(args)                                                                   │
│   2 _ = _build_t0_tokenizer(args)                                                                │
│ ❱ 3 _ = _build_knowledge_pool_in_bert(args)                                                      │
│   4 _ = _build_knowledge_pool_in_t0(args)                                                        │
│   5                                                                                              │
│                                                                                                  │
│ /mnt/data/ai4earth/zhangtianning/projects/llm/art/megatron/global_vars.py:531 in                 │
│ _build_knowledge_pool_in_bert                                                                    │
│                                                                                                  │
│   528 │   """Initialize T0 tokenizer."""                                                         │
│   529 │   global _GLOBAL_KNOWLEDGE                                                               │
│   530 │   assert 'bert' not in _GLOBAL_KNOWLEDGE                                                 │
│ ❱ 531 │   _GLOBAL_KNOWLEDGE['bert'] = Tokenized_Knowledge_Dataset(                               │
│   532 │   │   (args.indexed_evidence_bert_tokenized_data_path, args.indexed_title_bert_tokeniz   │
│   533 │   │    args.data_impl,                                                                   │
│   534 │   │   get_tokenizer(),args.seq_length_retriever                                          │
│                                                                                                  │
│ /mnt/data/ai4earth/zhangtianning/projects/llm/art/megatron/data/pretokenized_evidence.py:25 in   │
│ __init__                                                                                         │
│                                                                                                  │
│    22 │   │   indexed_evidence_bert_tokenized_data_path, indexed_title_bert_tokenized_data_pat   │
│    23 │   │   self.passages_map_bert = make_indexed_dataset(indexed_evidence_bert_tokenized_da   │
│    24 │   │   self.title_map_bert    = make_indexed_dataset(indexed_title_bert_tokenized_data_   │
│ ❱  25 │   │   print_rank_0(f'  >> total collect {len(self.passages_map_bert)} passages')         │
│    26 │   │   assert self.passages_map_bert is not None, f"you must provide the passage knowle   │
│    27 │   │   if mode == 'bert.both_query_and_passage':                                          │
│    28 │   │   │   assert self.title_map_bert is not None, f"you must provide the title pool fo   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: object of type 'NoneType' has no len()

In [3]:
#_ = _load_wikipedia_evidence(args)
_ = _build_t0_model(args)
_set_tensorboard_writer(args)
_set_timers()

  0%|          | 0/2 [00:01<?, ?it/s]

KeyboardInterrupt



In [3]:
initialize_megatron()

---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
> initializing torch distributed ...
---------> use fixed config <----------
> initializing model parallel with size 1
---------> use fixed config <----------
> initializing FAISS retriever groups
> setting random seeds to 1234 ...
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
---------> use fixed config <----------
---------> use fixed config <----------


In [3]:
from megatron.global_vars import get_tokenizer as get_bert_tokenizer
from tasks.dense_retriever.zero_shot_training.train_data_utils import OpenQADataset 

In [5]:
from tasks.dense_retriever.zero_shot_training.train_data_utils import *

In [6]:
from tasks.dense_retriever.zero_shot_training.train_data_utils import WhatIsQuestionDataset 
bert_tokenizer = get_bert_tokenizer()
args.train_data = ['data/unArXiv.key_word_from_abstract.csv']
train_dataset = WhatIsQuestionDataset("OPENQA_DATASET","training",args.train_data,bert_tokenizer,16)

---------> use fixed config <----------
 > building [What is?] dataset for [Judger~~~!]:
  > paths: data/unArXiv.key_word_from_abstract.csv
 > Processing data/unArXiv.key_word_from_abstract.csv ...


/mnt/data/ai4earth/zhangtianning/projects/llm/art/tasks/dense_retriever/zero_shot_training/train_data_utils.py:186: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)


 >> processed 1546447 samples.


In [7]:
train_dataset.samples[i+3]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_dataset.samples[i+3]                                                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'i' is not defined

In [8]:
from tqdm.notebook import tqdm
for i,b in tqdm(enumerate(train_dataset)):
    pass
    

0it [00:00, ?it/s]

In [7]:

train_dataset = OpenQADataset("OPENQA_DATASET","training",args.train_data,bert_tokenizer,16)

In [16]:
from megatron.model import ARTModel, PreComputedEvidenceDocsRetriever
from megatron.tokenizer import build_tokenizer
from megatron.model.art_model import JRTModel

### go train

In [10]:
os.environ["NCCL_DEBUG"] = "WARN"

In [10]:
from megatron.global_vars import get_t0_model, get_t0_tokenizer, get_knowledge_pool

In [11]:
from megatron.training import setup_model_and_optimizer
def model_provider():
        """Build the model."""
        args = get_args()
        print_rank_0('building ART model for {} ...'.format(args.task))
        evidence_retriever = PreComputedEvidenceDocsRetriever()
        model = JRTModel(evidence_retriever)
        return model

In [12]:
model, optimizer, lr_scheduler = setup_model_and_optimizer(model_provider)
model = model.module.module
model.evidence_retriever.local_rank = 0

---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
building ART model for ZERO-SHOT-RETRIEVER ...
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
we are going use random memory

> Building Brute Force MIPS index
>>> Finished adding block data to GPU index
NCCL version 2.14.3+cuda11.7
---------> use fixed config <----------
 > padded vocab (size: 30524) with 68 dummy tokens (new size: 30592)
building Retriever for ART (Autoencoding-based Retriever Training) ...
---------> use fixed config <----------
building DualEncoderModel...
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
---------> use fixed config <----------
-----

    will not load any checkpoints and will start from random


In [18]:
from tasks.dense_retriever.zero_shot_training.train import _build_train_dataloader
from tasks.dense_retriever.zero_shot_training.train import process_batch
from megatron.global_vars import get_t0_model, get_t0_tokenizer
import torch.nn.functional as F
import math

In [20]:
from tqdm.notebook import tqdm

In [21]:
train_dataloader = _build_train_dataloader(train_dataset)
for batch in tqdm(train_dataloader):
    pass

---------> use fixed config <----------
building training dataloader ...


  0%|          | 0/386615 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 train_dataloader = _build_train_dataloader(train_dataset)                                    │
│ ❱ 2 for batch in tqdm(train_dataloader):                                                         │
│   3 │   pass                                                                                     │
│   4                                                                                              │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/tqdm/notebook.py:254 in      │
│ __iter__                                                                                         │
│                                                                                                  │
│   251 │   def __iter__(self):                                                                    │
│   252 │   │   try:                                                                               │
│   253 │   │   │   it = super(tqdm_notebook, self).__iter__()                                     │
│ ❱ 254 │   │   │   for obj in it:                                                                 │
│   255 │   │   │   │   # return super(tqdm...) will not catch exception                           │
│   256 │   │   │   │   yield obj                                                                  │
│   257 │   │   # NB: except ... [ as ...] breaks IPython async KeyboardInterrupt                  │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/tqdm/std.py:1178 in __iter__ │
│                                                                                                  │
│   1175 │   │   time = self._time                                                                 │
│   1176 │   │                                                                                     │
│   1177 │   │   try:                                                                              │
│ ❱ 1178 │   │   │   for obj in iterable:                                                          │
│   1179 │   │   │   │   yield obj                                                                 │
│   1180 │   │   │   │   # Update and possibly print the progressbar.                              │
│   1181 │   │   │   │   # Note: does not call self.update(1) for speed optimisation.              │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader. │
│ py:633 in __next__                                                                               │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                            

In [11]:
fetcher= iter(train_dataloader)
batch = next(fetcher)

In [15]:
(query_uid, query_tokens_for_retriever, query_types, query_mask_bert,
               prefixed_query_token_for_LLM, prefixed_query_mask_t0,
               prefixed_query_token_len_for_LLM, reference) = process_batch(batch)

In [16]:
bert_tokenizer = get_bert_tokenizer()
t0_tokenizer = get_t0_tokenizer()

In [17]:
from megatron.data.mask_creation_utils import make_attention_mask

In [18]:
self = model
with torch.no_grad():
    query_embedding = self.get_query_embedding(query_tokens_for_retriever, query_mask_bert, query_types)
    topk_evidence_data = self.get_topk_evidence(query_uid, query_embedding)
    coarse_search_context_tokens_for_retriever, coarse_search_context_tokens_for_llm = self.evidence_retriever.postprocess(query_uid, topk_evidence_data,
                              prefixed_query_token_for_LLM)
    all_context_embedding = self.get_retriever_embedding(coarse_search_context_tokens_for_retriever)
    topk_log_probs        = self.compute_log_probs(query_embedding, all_context_embedding)
    prefixed_query_tuple  = (prefixed_query_token_for_LLM,prefixed_query_token_len_for_LLM)
    gold_log_probs = self.get_LLM_score(coarse_search_context_tokens_for_llm, prefixed_query_tuple)
    flattened_reference_ids = self.evidence_retriever.get_the_reference_id(topk_evidence_data)
    all_reference_embedding = self.get_retriever_embedding(flattened_reference_ids,embedder_type = "query")
    reference_sim = torch.einsum('bkd,bkd->bk',all_reference_embedding, all_context_embedding)
    reference_sim = reference_sim/(args.inverse_temperature_multiplier * math.sqrt(args.hidden_size))
    reference_sim = reference_sim.sigmoid() 


    reference_seted= torch.ones_like(gold_log_probs)

    retriever_score = torch.concatenate([topk_log_probs,reference_sim ],-1) #(B, 2*k)
    LLM_score       = torch.concatenate([gold_log_probs,reference_seted ],-1) #(B, 2*k)

---------> use fixed config <----------


In [35]:
torch.nn.BCELoss(reduction="mean")(retriever_score,LLM_score)

tensor(0.7261, device='cuda:0', dtype=torch.float16)

In [34]:
(gold_log_probs>0.5).float().mean()

tensor(0., device='cuda:0')

In [21]:
with torch.no_grad():
    

In [22]:
retriever_score.shape

torch.Size([4, 64])

In [23]:
LLM_score.shape

torch.Size([4, 64])

In [47]:
gold_log_probs

tensor([[0.1541, 0.0159, 0.0311, 0.1646, 0.0087, 0.0100, 0.0272, 0.0284, 0.0737,
         0.0260, 0.0230, 0.0077, 0.0185, 0.0119, 0.0197, 0.0077, 0.0489, 0.0268,
         0.0051, 0.0164, 0.0015, 0.0391, 0.0219, 0.0058, 0.0233, 0.0059, 0.0610,
         0.0147, 0.0967, 0.0099, 0.0077, 0.0213],
        [0.0637, 0.0592, 0.0421, 0.1481, 0.0839, 0.0362, 0.1688, 0.0592, 0.2510,
         0.0440, 0.0695, 0.1348, 0.1624, 0.0914, 0.1208, 0.0293, 0.0396, 0.0566,
         0.1561, 0.1943, 0.0421, 0.0336, 0.2147, 0.2227, 0.1442, 0.0666, 0.0792,
         0.1176, 0.1366, 0.1312, 0.0592, 0.0481],
        [0.0240, 0.0264, 0.0136, 0.1824, 0.0336, 0.0213, 0.0792, 0.0421, 0.0307,
         0.0293, 0.0534, 0.0284, 0.0049, 0.1848, 0.0995, 0.0194, 0.0352, 0.0503,
         0.0264, 0.3074, 0.0427, 0.0289, 0.0759, 0.1225, 0.1067, 0.0244, 0.0511,
         0.0040, 0.0268, 0.0197, 0.0145, 0.0526],
        [0.1176, 0.0090, 0.0147, 0.0427, 0.0264, 0.0244, 0.0174, 0.0122, 0.0191,
         0.0174, 0.0177, 0.0143, 0.0197,

In [54]:
model.llm_system.tokenizer.decode(output_ids.sequences[3])

'<unk><unk><unk><unk><unk><unk><unk><unk><unk><s> A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions. USER: Here is a passage about [ Antisemitism]:""" antisemitic—not just anti-Israel. Judith Popinski, an 86-year-old Holocaust survivor, stated that she is no longer invited to schools that have a large Muslim presence to tell her story of surviving the Holocaust. In December 2010, the Jewish human rights organization Simon Wiesenthal Center issued a travel advisory concerning Sweden, advising Jews to express "extreme caution" when visiting the southern parts of the country due to an alleged increase in verbal and physical harassment of Jewish citizens in the city of Malmö. Ilmar Reepalu, the mayor of Malmö for over 15 years, has been accused of failing to protect the Jewish""" Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed

In [41]:
t0_tokenizer.decode(coarse_search_context_tokens_for_llm[3])

'<s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions. USER: Given the passage:""" algebraically closed, because if "a", "a", …, "a" are the elements of "F", then the polynomial ("x" − "a")("x" − "a") ··· ("x" − "a") + 1 has no zero in "F". By contrast, the fundamental theorem of algebra states that the field of complex numbers is algebraically closed. Another example of an algebraically closed field is the field of (complex) algebraic numbers. Given a field "F", the assertion ""F" is algebraically closed" is equivalent to other assertions: The field "F" is algebraically closed if and only if the only irreducible polynomials in the polynomial ring "F"["x"] are those of""" Please check whether the passage is relative to the question: What is Lepton mass . Return Yes or No. ASSISTANT:'

In [42]:
real_answer_token = answer_token[:,3:]
if lm_logits.shape[1]<real_answer_token.shape[1]:
    lm_logits=torch.nn.functional.pad(lm_logits,(0,0,0,real_answer_token.shape[1] - lm_logits.shape[1]),mode ='replicate')
log_softmax    = F.log_softmax(lm_logits, dim=-1)
gold_log_probs = log_softmax.gather(2, real_answer_token.unsqueeze(2)).squeeze(2)


In [171]:
del model
torch.cuda.empty_cache()

In [36]:
log_softmax    = F.log_softmax(lm_logits, dim=-1)
gold_log_probs = alog_softmax.gather(2, answer_token.unsqueeze(2)).squeeze(2)

In [46]:
import transformers

In [47]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    'pretrain_weights/vicuna-7b-v1.1',
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token

In [48]:
_id = 0
# print(model.llm_system.tokenizer.decode(question_token[0]))
# print("==================================================")
# print(model.llm_system.tokenizer.decode(context_tensor[_id]))
# print("==================================================")
#print(model.llm_system.tokenizer.decode(output_ids.sequences[_id]))
#print("==================================================")
print(model.llm_system.tokenizer.decode(lm_logits.max(-1)[1][_id]))
print(model.llm_system.tokenizer.decode(answer_token[_id][3:]))

 
. with two friend person and a expert intelligence..
 user provides a advice inform answers and accurateite responses to the user'<s> questions.
ER</s>I is a list from theobjectosism-Capitalism](
An
ud, thelary, theft, murder ev are
 stateiation of physical is the considered to as aggression, violenceercion.
 idea between anarcho-capitalism and min libertarians is that one of emphas means of which they believe the principleiom seriously Anarchists libertarians, for as the members who in thearian parties parties, believe limit a state as order form form more intrive form, suching only least most least the order and courts, defense. An, an, argue the argumentsances for the functions functions,
 an, anarcho-capitalists argue the and of state intervention, including the state as an violercive monopoly on advocby such use institution that society history" history on this passage passage, which a a question for cans with closelyately with the main purpose thatsem in it passage.
.
an Answer

In [290]:
from transformers import AutoTokenizer,AutoModelForCausalLM
from fastchat.conversation import get_conv_template
tokenizer = AutoTokenizer.from_pretrained("pretrain_weights/vicuna-7b-v1.1", use_fast=False)
conv = get_conv_template('vicuna_v1.1').copy()
conv.append_message(conv.roles[0], 'Here is a passage about [Anatoly Karpov]:"""A rematch was set for later in 1985, also in Moscow. The events of the so-called Marathon Match forced FIDE to return to the previous format, with a match limited to 24 games (with Karpov remaining champion if the match should finish 12–12). Karpov needed to win the final game to draw the match and retain his title, but wound up losing, thus surrendering the title to his opponent. The final score was 13–11 (+3−5=16), in favour of Kasparov. Karpov remained a formidable opponent (and the world No. 2) until the early 1990s. He fought Kasparov in three more world"""Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed within the text.')
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
input_ids = tokenizer([prompt]).input_ids

In [173]:
model.llm_system.tokenizer.decode(input_ids[0])

'<s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions. USER: Here is a passage about [Anatoly Karpov]:"""A rematch was set for later in 1985, also in Moscow. The events of the so-called Marathon Match forced FIDE to return to the previous format, with a match limited to 24 games (with Karpov remaining champion if the match should finish 12–12). Karpov needed to win the final game to draw the match and retain his title, but wound up losing, thus surrendering the title to his opponent. The final score was 13–11 (+3−5=16), in favour of Kasparov. Karpov remained a formidable opponent (and the world No. 2) until the early 1990s. He fought Kasparov in three more world"""Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed within the text. ASSISTANT:'

In [295]:
with torch.no_grad():
    output_ids = model.llm_system.model.generate(torch.as_tensor(input_ids).cuda(), return_dict_in_generate=False, output_scores=False,max_length=512)
    print(model.llm_system.tokenizer.decode(input_ids[0]))
    print("------------------------------------------------")
    print(model.llm_system.tokenizer.decode(output_ids[0]))

<s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Here is a passage about [Anatoly Karpov]:"""A rematch was set for later in 1985, also in Moscow. The events of the so-called Marathon Match forced FIDE to return to the previous format, with a match limited to 24 games (with Karpov remaining champion if the match should finish 12–12). Karpov needed to win the final game to draw the match and retain his title, but wound up losing, thus surrendering the title to his opponent. The final score was 13–11 (+3−5=16), in favour of Kasparov. Karpov remained a formidable opponent (and the world No. 2) until the early 1990s. He fought Kasparov in three more world"""Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed within the text. ASSISTANT:
------------------------------------------------
<s> A chat between a curious user and 

In [15]:
# model.module.module.evidence_retriever.local_rank = 0
# with torch.no_grad():
#     topk_log_probs, gold_log_probs = model(query_uid,
#                                            query_ids_bert,
#                                            query_types,
#                                            query_mask_bert,
#                                            prefixed_query_ids_t0,
#                                            prefixed_query_ids_t0_len)

dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Using network Socket
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Channel 00/02 :    0
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Channel 01/02 :    0
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Trees [0] -1/-1/-1->0->-1 [1] -1/-1/-1->0->-1
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Connected all rings
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO Connected all trees
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO 2 coll channels, 2 p2p channels, 2 p2p channels per peer
dsw-23976-7d79d88dc8-j998v:63957:64669 [0] NCCL INFO comm 0x77bc6bc0 rank 0 nranks 1 cudaDev 0 busId 70 - Init COMPLETE
---------> use fixed config <----------


In [18]:
retriever_loss = torch.FloatTensor([0]).cuda()
if args.update_retriever:
    topk_log_probs = topk_log_probs.float()
    gold_log_probs = gold_log_probs.float()
    gold_log_probs_log_softmax = F.log_softmax(gold_log_probs, dim=1)
    loss_func = torch.nn.KLDivLoss(reduction="batchmean", log_target=True)
    retriever_loss = loss_func(topk_log_probs, gold_log_probs_log_softmax)

net_loss = retriever_loss

##### the new model

In [16]:
model = model.module.module
model.evidence_retriever.local_rank= 0

In [30]:
with torch.no_grad():

    #args = get_args()
    args = model.args
    # assert bsize == 1, "for auto-encoder pre-training, we assume a local batch size of 1"
    assert args.initialize_t0_model_tokenizer_evidence, "for auto-encoder pre-training, we need to pass the argument --initialize-t0-model-and-tokenizer"
    query_logits =  model.get_query_embedding(query_ids_bert, query_mask_bert, query_types)
    all_title_context_ids_bert_tokenized, all_title_context_ids_for_t0 = model.get_topk_evidence(query_uid, query_logits)
    # Compute the retriever score
    all_context_logits = model.get_answer_embedding(all_title_context_ids_bert_tokenized)
    topk_log_probs     = model.compute_log_probs(query_logits,all_context_logits)
    # Compute the language model score of the retrieved passages
    prefixed_query_tuple = (prefixed_query_ids_t0,prefixed_query_ids_t0_len)
    gold_log_probs = model.get_LLM_score(all_title_context_ids_for_t0, prefixed_query_tuple)

---------> use fixed config <----------


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 │   topk_log_probs     = model.compute_log_probs(query_logits,all_context_logits)           │
│   12 │   # Compute the language model score of the retrieved passages                            │
│   13 │   prefixed_query_tuple = (prefixed_query_ids_t0,prefixed_query_ids_t0_len)                │
│ ❱ 14 │   gold_log_probs = model.get_LLM_score(all_title_context_ids_for_t0, prefixed_query_tu    │
│   15                                                                                             │
│                                                                                                  │
│ /mnt/data/ai4earth/zhangtianning/projects/llm/art/megatron/model/art_model.py:133 in             │
│ get_LLM_score                                                                                    │
│                                                                                                  │
│   130 │   │   │   │   all_title_context_ids_view = all_title_context_ids_one_question[i: i + a   │
│   131 │   │   │   │   decoder_prefix_tensor_view = decoder_prefix_tensor_one_question[i: i + a   │
│   132 │   │   │   │   with torch.no_grad():                                                      │
│ ❱ 133 │   │   │   │   │   gold_log_probs = self.llm_system.get_gold_log_probs(all_title_contex   │
│   134 │   │   │   │   │   # this will work because the batch size is 1 and this implies all de   │
│   135 │   │   │   │   │   teacher_log_probs = torch.mean(gold_log_probs[:, :prefixed_query_ids   │
│   136 │   │   │   │   │   log_prob_list_one_question.append(teacher_log_probs)                   │
│                                                                                                  │
│ /mnt/data/ai4earth/zhangtianning/projects/llm/art/megatron/model/art_model.py:33 in              │
│ get_gold_log_probs                                                                               │
│                                                                                                  │
│    30 │   │   │   │   │   │   │   │   │   │   return_tensors='pt')                               │
│    31 │   │   assert input_encoding.input_ids.size(1) <= 512                                     │
│    32 │   │   context_tensor, attention_mask = input_encoding.input_ids.cuda(), input_encoding   │
│ ❱  33 │   │   lm_output = self.model(input_ids=context_tensor,attention_mask=attention_mask,la   │
│    34 │   │   │   │   │    output_attentions=False,output_hidden_states=False)                   │
│    35 │   │   if isinstance(lm_output,tuple):                                                    │
│    36 │   │   │   ### the llama return                                                           │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:1 │
│ 501 in _call_impl                                                                                │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks

In [29]:
torch.cuda.empty_cache()

In [24]:
ainput_encoding.input_ids.shape

torch.Size([16, 272])

In [ ]:
input_encoding = self.tokenizer.pad({'input_ids': question_token},
                                padding='longest',
                                max_length=512,
                                pad_to_multiple_of=8,
                                return_attention_mask=True,
                                return_tensors='pt')
assert input_encoding.input_ids.size(1) <= 512
context_tensor, attention_mask = input_encoding.input_ids.cuda(), input_encoding.attention_mask.cuda()
lm_output = self.model(input_ids=context_tensor,
                            attention_mask=attention_mask,
                            labels=answer_token,
                            output_attentions=False,
                            output_hidden_states=False)
lm_logits = lm_output.logits.float()
log_softmax = F.log_softmax(lm_logits, dim=-1)
gold_log_probs = log_softmax.gather(2, answer_token.unsqueeze(2)).squeeze(2)

In [16]:
model.llm_system.tokenizer.decode(all_title_context_ids_for_t0[0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model.llm_system.tokenizer.decode(all_title_context_ids_for_t0[0])                           │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'all_title_context_ids_for_t0' is not defined

In [64]:
model.llm_system.tokenizer.decode(topk_evidence_data[0][2][0][1])

'Economy of Azerbaijan'

In [24]:
bert_tokenizer.decode(all_title_context_ids_bert_tokenized[0])

'[CLS] us $ 1 ) for 28 january 2016, was azn 1. 60. there is a complex relationship between azerbaijan\'s balance of trade, inflation, measured by the consumer price index and the value of its currency. despite allowing the value of the manat to " float ", azerbaijan\'s central bank has decisive ability to control its value with relationship to other currencies. two thirds of azerbaijan is rich in oil and natural gas. the region of the lesser caucasus accounts for most of the country\'s gold, silver, iron, copper, titanium, chromium, manganese, cobalt, molybdenum, complex ore and antimony. in september 1994, a 30 - year contract was signed between [SEP] economy of azerbaijan [SEP]'

In [73]:
len(query_ids_bert)

4

In [76]:
bert_tokenizer.decode(query_ids_bert[3].tolist())

'[CLS] who has the most turnovers in nba history [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [58]:
bert_tokenizer.decode(topk_evidence_data[0][1][0][1])

'economy of azerbaijan'

##### the original model

In [ ]:
from megatron.model.art_model import postprocess

In [51]:
args = get_args()
topk = model.topk
bsize, max_seq_len = query_ids_bert.shape

# assert bsize == 1, "for auto-encoder pre-training, we assume a local batch size of 1"
assert args.initialize_t0_model_tokenizer_evidence, "for auto-encoder pre-training, we need to pass the argument --initialize-t0-model-and-tokenizer"

# Compute "fresh" query logits
query_logits = model.retriever_embedder(query_ids_bert,
                                        query_mask_bert,
                                        query_types,
                                        embedder_type="query",
                                        disable_dropout=args.disable_retriever_dropout)
if args.no_query_embedder_training:
    query_logits = query_logits.detach()

# Get top-K evidence data for the BERT tokenized query
with torch.no_grad():
    topk_evidence_data, stale_topk_sim = model.evidence_retriever.get_topk(query_logits.clone().detach())

---------> use fixed config <----------


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 # Compute the language model score of the retrieved passages                                │
│    9 prefixed_query_tuple = (prefixed_query_ids_t0,                                              │
│   10 │   │   │   │   │   │   prefixed_query_ids_t0_len)                                          │
│ ❱ 11 gold_log_probs = model.get_LLM_score(                                                       │
│   12 │   all_title_context_ids_for_t0, prefixed_query_tuple)                                     │
│   13 # Compute the retriever score                                                               │
│   14 topk_log_probs = model.get_fine_search_result(                                              │
│                                                                                                  │
│ in get_LLM_score:126                                                                             │
│                                                                                                  │
│   123 │   │   │   │   decoder_prefix_tensor_view = decoder_prefix_tensor_one_question[i: i + a   │
│   124 │   │   │   │                                                                              │
│   125 │   │   │   │   with torch.no_grad():                                                      │
│ ❱ 126 │   │   │   │   │   lm_output = language_model(input_ids=context_tensor,                   │
│   127 │   │   │   │   │   │   │   │   │   │   │      attention_mask=attention_mask,              │
│   128 │   │   │   │   │   │   │   │   │   │   │      labels=decoder_prefix_tensor_view,          │
│   129 │   │   │   │   │   │   │   │   │   │   │      output_attentions=False,                    │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:1 │
│ 501 in _call_impl                                                                                │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/t5/model │
│ ing_t5.py:1679 in forward                                                                        │
│                                                                                                  │
│   1676 │   │   # Encode if needed (training, first prediction pass)                              │
│   1677 │   │   if encoder_outputs is None:                                                       │
│   1678 │   │   │   # Convert encoder inputs in embeddings if needed                              │
│ ❱ 1679 │   │   │   encoder_outputs = self.encoder(                                               │
│   1680 │   │   │   │   input_ids=input_ids,                

In [64]:
with torch.no_grad():
    output = postprocess(query_uid,topk_evidence_data)
    all_title_context_ids_bert_tokenized, all_title_context_ids_for_t0 = output

---------> use fixed config <----------
---------> use fixed config <----------


In [53]:
from megatron.model.art_model import get_tokenizer,get_t0_tokenizer

In [67]:
bert_tokenizer = get_tokenizer()
t0_tokenizer   = get_t0_tokenizer()

In [138]:
# Compute the language model score of the retrieved passages
decoder_prefix_tensor = torch.repeat_interleave(prefixed_query_ids_t0, topk, dim=0)
log_prob_list = []

In [139]:
from megatron import get_t0_model
import torch.nn.functional as F

In [140]:
language_model = get_t0_model()
language_model = language_model.cuda()

## 按照每个 batch 来进行 inference
for k in range(0, bsize * topk, topk):
    log_prob_list_one_question = []
    all_title_context_ids_one_question     = all_title_context_ids_for_t0[k: k + topk]
    decoder_prefix_tensor_one_question     = decoder_prefix_tensor[k: k + topk]
    prefixed_query_ids_t0_len_one_question = prefixed_query_ids_t0_len[k // topk]

    ### 主要为了控制中间的计算消耗
    for i in range(0, topk, args.shard_size):
        all_title_context_ids_view = all_title_context_ids_one_question[i: i + args.shard_size]
        # pad the sequences
        input_encoding = model.t0_tokenizer.pad({'input_ids': all_title_context_ids_view},
                                                  padding='longest',
                                                  max_length=512,
                                                  pad_to_multiple_of=8,
                                                  return_attention_mask=True,
                                                  return_tensors='pt')
        assert input_encoding.input_ids.size(1) <= 512
        context_tensor, attention_mask = input_encoding.input_ids.cuda(), input_encoding.attention_mask.cuda()
        decoder_prefix_tensor_view     = decoder_prefix_tensor_one_question[i: i + args.shard_size]

        with torch.no_grad():
            lm_output = language_model(input_ids           =context_tensor,
                                       attention_mask      =attention_mask,
                                       labels              =decoder_prefix_tensor_view,
                                       output_attentions   =False,
                                       output_hidden_states=False)
            lm_logits = lm_output.logits.float()
            _, decoder_seq_length, vocab_size = lm_logits.shape

            log_softmax    = F.log_softmax(lm_logits, dim=-1)
            # pick up the answer's log softmax value
            # gather 第一行取哪几个， 第二行取哪几个
            gold_log_probs = log_softmax.gather(2, decoder_prefix_tensor_view.unsqueeze(2)).squeeze(2)

            # this will work because the batch size is 1 and this implies all decoder labels have the same length
            teacher_log_probs = torch.mean(gold_log_probs[:, :prefixed_query_ids_t0_len_one_question], dim=1)
            log_prob_list_one_question.append(teacher_log_probs)
    log_prob_list_one_question = torch.cat(log_prob_list_one_question).unsqueeze(0)
    log_prob_list.append(log_prob_list_one_question)


gold_log_probs = torch.cat(log_prob_list, dim=0)

# Compute the retriever score
input_encoding = model.hf_bert_tokenizer.pad({'input_ids': all_title_context_ids_bert_tokenized},
                                            padding='longest',
                                            max_length=512,
                                            pad_to_multiple_of=8,
                                            return_attention_mask=True,
                                            return_tensors='pt')
assert input_encoding.input_ids.size(1) <= 512

all_title_context_ids = input_encoding.input_ids.cuda()
all_context_types     = torch.cuda.LongTensor(input_encoding.input_ids.size()).fill_(0)
all_context_mask      = (all_title_context_ids[:, None, :] >= 1) * (all_title_context_ids[:, :, None] >= 1)

# Inverting the mask
all_context_mask      = ~all_context_mask

# Compute "fresh" context logits
all_context_logits = model.retriever_embedder(all_title_context_ids,
                                                all_context_mask,
                                                all_context_types,
                                                embedder_type="context",
                                                disable_dropout=args.disable_retriever_dropout)
all_context_logits = all_context_logits.reshape(bsize, topk, -1)

if args.no_context_embedder_training:
    all_context_logits = all_context_logits.detach()
query_logits       = query_logits.unsqueeze(1).float()
all_context_logits = all_context_logits.float()

# [B, 1, K]
topk_sim_scores    = torch.bmm(query_logits, all_context_logits.transpose(1, 2))
if args.retriever_score_scaling:
    topk_sim_scores = topk_sim_scores / (args.inverse_temperature_multiplier * math.sqrt(args.hidden_size))
# [B, 1, K]
topk_log_probs = F.log_softmax(topk_sim_scores, dim=2)
# B x 1 x K -> B x K
topk_log_probs = topk_log_probs.squeeze(1)

In [142]:
import math

In [15]:
model.module.module.evidence_retriever.local_rank = 0

In [16]:
# train_iters, gold_log_probs = model(query_uid,
#                                            query_ids_bert,
#                                            query_types,
#                                            query_mask_bert,
#                                            prefixed_query_ids_t0,
#                                            prefixed_query_ids_t0_len)

---------> use fixed config <----------
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Using network Socket
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Channel 00/02 :    0
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Channel 01/02 :    0
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Trees [0] -1/-1/-1->0->-1 [1] -1/-1/-1->0->-1
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Connected all rings
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO Connected all trees
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO 2 coll channels, 2 p2p channels, 2 p2p channels per peer
dsw-23976-7d79d88dc8-j998v:8547:8946 [0] NCCL INFO comm 0x786eea90 rank 0 nranks 1 cudaDev 0 busId 70 - Init COMPLETE
---------> use fixed config <----------


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_iters, gold_log_probs = model(query_uid,                                               │
│   2 │   │   │   │   │   │   │   │   │   │      query_ids_bert,                                   │
│   3 │   │   │   │   │   │   │   │   │   │      query_types,                                      │
│   4 │   │   │   │   │   │   │   │   │   │      query_mask_bert,                                  │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:1 │
│ 501 in _call_impl                                                                                │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /mnt/data/ai4earth/zhangtianning/projects/llm/art/megatron/model/distributed.py:76 in forward    │
│                                                                                                  │
│    73 │                                                                                          │
│    74 │   def forward(self, *inputs, **kwargs):                                                  │
│    75 │   │   self.needs_reduction = True                                                        │
│ ❱  76 │   │   return self.module(*inputs, **kwargs)                                              │
│    77 │                                                                                          │
│    78 │   def state_dict(self, destination=None, prefix='', keep_vars=False):                    │
│    79 │   │   #[h.remove() for h in self.hook_handles]                                           │
│                                                                                                  │
│ /home/zhangtianning/.conda/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py:1 │
│ 501 in _call_impl                                                                                │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                            

In [9]:
train_iters


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_iters                                                                                  │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_iters' is not defined